In [1]:
from multisensory_playbook import (
    ClassicalTask,
    DetectionTask,
    DetectionTask_versatile,
    LinearClassifier,
    Trials
)

import pickle
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from scipy.optimize import fsolve
import joblib
import sys, os
from pylab import *

from RNN_helpers import build_input_layer, batch_generator_rnn, train_rnn, check_training_loss_anomalies
def calculate_pg(ff, k, N=90, correction=1):
    """
    Use ff to calulate pg, given a k and N
    ff: desired filtered fraction of E 
    pg: probability of E(t)=1 in the base_e (generator)
    k : local on-time duration
    N : number time-steps
    """
    buffer = k
    #pg = (1-fsolve(lambda x: ff-(1-x**k)/(1-x**(N)), 0.9))[0] 
    if correction:
        ff = (1-fsolve(lambda x: ff-(1-x**k)/(1-x**(N+int(buffer)-1*(k-1))), 0.9))[0]

    return ff

def levy_solve_for_pg(ff_desired, pl, N=500): 
    cpl=cumsum(pl) 

    lmax = len(pl)-1 
    
    lengths = arange(lmax+1) 
    
    fs = lambda pg: 1-prod(1-pg+pg*cpl) 
    
    ff = lambda pg: fs(pg)/(1-prod((1-pg*pl[1:])**(N+lengths[1:]-1))) 
    
    pg = fsolve(lambda pg: ff_desired-ff(pg), ff_desired)[0] 
    
    return pg 


def levy_dist(lmax):
  l = arange(lmax+1) # allow l=0 but must have p=0
  pl = zeros(lmax+1)
  pl[1:] = 1.0/l[1:]**2
  pl[:] /= sum(pl)
  return pl
    
def generate_mix_samples(pg, pl, N=90, repeats=10):
  
  lmax = len(pl)-1
  #print('lmax ', lmax)
  #pgl = pg*pl
  M = N+lmax-1
  all_E = []
  positions = arange(M)
  lengths = arange(lmax+1)
  for _ in range(repeats):
    keep_going = True
    while keep_going:
      # generate nonzero points
      num_nonzero = binomial(M, pg)
      if num_nonzero==0:
        continue
      E_starts = choice(positions, size=num_nonzero, replace=False)
      L = choice(lengths, size=num_nonzero, p=pl)
      #print('L ', L)
      E = zeros(M, dtype=bool)
      for e_start, l in zip(E_starts, L):
        E[e_start:e_start+l] = 1
      E = E[lmax-1:]
      assert len(E)==N
      keep_going = (sum(E)==0)
    all_E.append(E)
  all_E = array(all_E) # shape (repeats, N)
  return all_E

def estimate_fraction_on(pg, pl, N=90, repeats=1000):
  return generate_mix_samples(pg, pl, N=N, repeats=repeats).mean()

def generate_levy_AV(pm, pn, pi, pc, nb_trials, nb_steps, E):
    arr_M = choice([-1, 0, 1], size=nb_trials, p=[pm / 2, 1 - pm, pm / 2])
    arr_A = np.zeros((nb_trials, nb_steps), dtype=int) # Q1! + k padding in the begining of E for Levy flights?
    arr_V = np.zeros((nb_trials, nb_steps), dtype=int)
    arr_E = E #np.zeros((nb_trials, nb_steps-k), dtype=int)

    for trial in range(nb_trials):
        M = arr_M[trial]
        e0 = np.array([-1, 0, 1]) # Add noise if E = 0
        p_e0 = np.array([pn / 2, 1 - pn, pn / 2])
        e1 = np.array([-M, 0, M]) # add probabilities for incorrectness
        p_e1 = np.array([pi, 1 + (- pc - pi), pc])
        
        A = np.where(E[trial], choice(e1, size=E[trial].size, p=p_e1), choice(e0, size=E[trial].size, p=p_e0))
        V = np.where(E[trial], choice(e1, size=E[trial].size, p=p_e1), choice(e0, size=E[trial].size, p=p_e0))
        arr_A[trial, :] = A 
        arr_V[trial, :] = V
        #arr_E[trial, :] = E[trial]
            
    return arr_M, arr_A, arr_V,arr_E

# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available! GPU support is enabled.")
else:
    print("CUDA is not available. Please check your installation.")

# Set the default tensor type to CUDA tensors if CUDA is available
if torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
    torch.set_default_device('cuda')

# Number of GPUs available
num_gpus = torch.cuda.device_count()
#print(f"Number of GPUs available: {num_gpus}")

# Details of each GPU
for i in range(num_gpus):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
    print(f"  Memory Allocated: {torch.cuda.memory_allocated(i)} bytes")
    print(f"  Memory Cached: {torch.cuda.memory_reserved(i)} bytes")
    print(f"  Compute Capability: {torch.cuda.get_device_properties(i).major}.{torch.cuda.get_device_properties(i).minor}")

# Network parameters (must match the trained network)
nb_inputs = 4
nb_hidden = 100
nb_outputs = 3
nb_steps = 500
batch_size = 32
nb_trials = batch_size * 500

# Task parameters
time_dep = 1
pe_sparse = 0.04
pm, pn, pi, pc = 1, 1/3, 0.01, 0.45

CUDA is available! GPU support is enabled.
GPU 0: NVIDIA RTX A6000
  Memory Allocated: 0 bytes
  Memory Cached: 0 bytes
  Compute Capability: 8.6


/home/jupyter-swathi/.conda/envs/cuda-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/jupyter-swathi/.conda/envs/cuda-env/lib/python3.12/site-packages/torch/__init__.py:747: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at /opt/conda/conda-bld/pytorch_1712608847532/work/torch/csrc/tensor/python_tensor.cpp:431.)
  _C._set_default_tensor_type(t)


### Train on levy/flat test on levy/flat

In [20]:
# Initialize model architecture
def model(input_data, rnn_layer, linear_layer):
    input_data = input_data.cuda()
    batchsize = input_data.shape[1]
    h0 = torch.zeros((1, batchsize, nb_hidden), dtype=torch.float32).cuda()
    out, h1 = rnn_layer(input_data, h0)
    lin_out = linear_layer(out)
    sum_out = lin_out.sum(axis=0).cuda()
    return sum_out
# Dictionary to store test accuracies
test_acc_list = []

# Create test tasks
lmax = 8 #levy k max
pl = levy_dist(lmax) #  8 * [1/8] #

E = generate_mix_samples(pg=levy_solve_for_pg(pe_sparse, pl, N=500), pl=pl, N=nb_steps, repeats=nb_trials)

# Generate testing data 
training_size = nb_trials
M, A, V, E = generate_levy_AV(pm, pn, pi, pc, nb_trials, nb_steps, E)
testing_trials = Trials(
    repeats=training_size,
    time_steps=nb_steps+lmax-1,
    M=M,
    A=A,
    V=V,
    task=None
)
# Path to saved model (modify as needed)
for seed in [1000, 2000, 3000, 4000, 5000]:  # specify your seed
    path = f"./RNN/RNN_train_onlevy_seed_{seed}_{nb_steps}steps"

    # Prepare test data
    y_test, A_test, V_test = torch.tensor(testing_trials.M + 1), testing_trials.A, testing_trials.V
    
    # Create fresh network instances
    test_rnn_layer = nn.RNN(input_size=nb_inputs, hidden_size=nb_hidden, 
                            nonlinearity='relu', batch_first=False).cuda()
    test_linear_layer = nn.Linear(nb_hidden, nb_outputs).cuda()
    
    # Load the trained model
    checkpoint = torch.load(path + '_checkpoint.pkl')
    test_rnn_layer.load_state_dict(checkpoint['rnn_layer_state_dict'])
    test_linear_layer.load_state_dict(checkpoint['linear_layer_state_dict'])
    
    # Set to evaluation mode
    test_rnn_layer.eval()
    test_linear_layer.eval()
    
    # Test the model
    with torch.no_grad():
        input_layer = build_input_layer(A_test, V_test)
        y_pred = model(input_layer, test_rnn_layer, test_linear_layer).cpu().numpy()
        y_pred_argmax = np.argmax(y_pred, axis=1)
        test_acc = (np.mean(y_pred_argmax == y_test.cpu().numpy())) * 100
        test_acc_list.append(test_acc)
        #print(f"Test accuracy for k={test_k}: {test_acc:.2f}%")
    
    # Clear memory
    del test_rnn_layer, test_linear_layer
    del input_layer, y_pred, y_pred_argmax
    torch.cuda.empty_cache()

# Save the test accuracies
#with open(path + '_testaccuracies_loadtest.pkl', 'wb') as file:
#    pickle.dump(test_acc_dict, file)

print("Testing complete. Results saved.")

Testing complete. Results saved.


In [22]:
np.save('RNN_testonlevy_trainonlevy.npy', test_acc_list)

### Train on k test on levy/flat

In [33]:
# Initialize model architecture
def model(input_data, rnn_layer, linear_layer):
    input_data = input_data.cuda()
    batchsize = input_data.shape[1]
    h0 = torch.zeros((1, batchsize, nb_hidden), dtype=torch.float32).cuda()
    out, h1 = rnn_layer(input_data, h0)
    lin_out = linear_layer(out)
    sum_out = lin_out.sum(axis=0).cuda()
    return sum_out
# Dictionary to store test accuracies
test_acc_list = {}

# Create test tasks
lmax = 8 #levy k max
pl =  levy_dist(lmax) # 8 * [1/8] #

E = generate_mix_samples(pg=levy_solve_for_pg(pe_sparse, pl, N=500), pl=pl, N=nb_steps, repeats=nb_trials)

# Generate testing data 
training_size = nb_trials
M, A, V, E = generate_levy_AV(pm, pn, pi, pc, nb_trials, nb_steps, E)
testing_trials = Trials(
    repeats=training_size,
    time_steps=nb_steps+lmax-1,
    M=M,
    A=A,
    V=V,
    task=None
)
# Path to saved model (modify as needed)
for k in range(1,9):
    print(k)
    acclist = []
    for seed in [1000, 2000, 3000, 4000, 5000]:  # specify your seed
        path = f"./data/RNN_trainedcheckpoints/RNN_train_k_{k}_seed_{seed}_{nb_steps}steps"
    
        # Prepare test data
        y_test, A_test, V_test = torch.tensor(testing_trials.M + 1), testing_trials.A, testing_trials.V
        
        # Create fresh network instances
        test_rnn_layer = nn.RNN(input_size=nb_inputs, hidden_size=nb_hidden, 
                                nonlinearity='relu', batch_first=False).cuda()
        test_linear_layer = nn.Linear(nb_hidden, nb_outputs).cuda()
        
        # Load the trained model
        checkpoint = torch.load(path + '_checkpoint.pkl')
        test_rnn_layer.load_state_dict(checkpoint['rnn_layer_state_dict'])
        test_linear_layer.load_state_dict(checkpoint['linear_layer_state_dict'])
        
        # Set to evaluation mode
        test_rnn_layer.eval()
        test_linear_layer.eval()
        
        # Test the model
        with torch.no_grad():
            input_layer = build_input_layer(A_test, V_test)
            y_pred = model(input_layer, test_rnn_layer, test_linear_layer).cpu().numpy()
            y_pred_argmax = np.argmax(y_pred, axis=1)
            test_acc = (np.mean(y_pred_argmax == y_test.cpu().numpy())) * 100
            acclist.append(test_acc)
            #print(f"Test accuracy for k={test_k}: {test_acc:.2f}%")
        print(test_acc)
        # Clear memory
        del test_rnn_layer, test_linear_layer
        del input_layer, y_pred, y_pred_argmax
        torch.cuda.empty_cache()
    test_acc_list[k] = acclist
# Save the test accuracies
#with open(path + '_testaccuracies_loadtest.pkl', 'wb') as file:
#    pickle.dump(test_acc_dict, file)

print("Testing complete. Results saved.")

1
82.65
82.5875
82.39999999999999
82.44375000000001
82.49375
2
82.99375
82.96249999999999
82.78750000000001
82.825
83.18124999999999
3
83.61874999999999
83.55
83.30625
83.49375
83.525
4
83.04375
83.2375
83.625
83.5125
83.1375
5
82.94375000000001
83.25
83.40625
83.25625
83.53125
6
83.35625
83.23125
82.79375
83.1125
83.26249999999999
7
83.275
82.70625
82.66875
82.48124999999999
82.45
8
82.525
82.7125
81.50625000000001
82.35625
81.66875
Testing complete. Results saved.


In [35]:
np.save('RNN_testonlevy_trainonk.npy', test_acc_list, allow_pickle=True) 

In [34]:
test_acc_list

{1: [82.65, 82.5875, 82.39999999999999, 82.44375000000001, 82.49375],
 2: [82.99375,
  82.96249999999999,
  82.78750000000001,
  82.825,
  83.18124999999999],
 3: [83.61874999999999, 83.55, 83.30625, 83.49375, 83.525],
 4: [83.04375, 83.2375, 83.625, 83.5125, 83.1375],
 5: [82.94375000000001, 83.25, 83.40625, 83.25625, 83.53125],
 6: [83.35625, 83.23125, 82.79375, 83.1125, 83.26249999999999],
 7: [83.275, 82.70625, 82.66875, 82.48124999999999, 82.45],
 8: [82.525, 82.7125, 81.50625000000001, 82.35625, 81.66875]}